# RNN을 이용하여 텍스트 생성하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기: %d" % vocab_size)

단어 집합의 크기: 12


In [ ]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [ ]:
sequences = list()
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수 %d' % len(sequences))
print(sequences)

학습에 사용할 샘플의 개수 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [ ]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이: {}'.format(max_len))

샘플의 최대 길이: 6


In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') # 문장 길이 정해두고 앞쪽에 패딩 넣기
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [ ]:
sequences = np.array(sequences)
X = sequences[:, :-1] # 마지막 항목(=y; 예측 대상)만 떼어내서 분리
y = sequences[:, -1]
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [ ]:
y = to_categorical(y, num_classes=vocab_size) # 정답을 원-핫 인코딩
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # embedding layer
model.add(SimpleRNN(hidden_units)) # hidden layer
model.add(Dense(vocab_size, activation='softmax')) # output layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 3s - 3s/step - accuracy: 0.0000e+00 - loss: 2.4782
Epoch 2/200
1/1 - 0s - 208ms/step - accuracy: 0.1818 - loss: 2.4622
Epoch 3/200
1/1 - 0s - 56ms/step - accuracy: 0.1818 - loss: 2.4461
Epoch 4/200
1/1 - 0s - 45ms/step - accuracy: 0.2727 - loss: 2.4299
Epoch 5/200
1/1 - 0s - 49ms/step - accuracy: 0.3636 - loss: 2.4133
Epoch 6/200
1/1 - 0s - 54ms/step - accuracy: 0.3636 - loss: 2.3964
Epoch 7/200
1/1 - 0s - 58ms/step - accuracy: 0.4545 - loss: 2.3790
Epoch 8/200
1/1 - 0s - 53ms/step - accuracy: 0.5455 - loss: 2.3609
Epoch 9/200
1/1 - 0s - 54ms/step - accuracy: 0.4545 - loss: 2.3422
Epoch 10/200
1/1 - 0s - 57ms/step - accuracy: 0.4545 - loss: 2.3228
Epoch 11/200
1/1 - 0s - 136ms/step - accuracy: 0.4545 - loss: 2.3024
Epoch 12/200
1/1 - 0s - 56ms/step - accuracy: 0.4545 - loss: 2.2812
Epoch 13/200
1/1 - 0s - 50ms/step - accuracy: 0.4545 - loss: 2.2590
Epoch 14/200
1/1 - 0s - 59ms/step - accuracy: 0.4545 - loss: 2.2358
Epoch 15/200
1/1 - 0s - 56ms/step - accuracy: 0.4545 

In [ ]:
# 예측

def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    print('init_word:', init_word)
    return sentence

In [ ]:
print(sentence_generation(model, tokenizer, '경마장에', 3))

init_word: 경마장에
경마장에 있는 말이 뛰고


# LSTM을 이용하여 텍스트 생성하기

In [1]:
# 뉴욕 타임즈 기사의 제목을 데이터셋으로 예측하기

import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 개수:', len(df.columns))
print(df.columns)

열의 개수: 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
print(df['headline'].isnull().values.any()) # 제목에 해당하는 부분이 null 이 있는지 확인

False


In [6]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values))
headline[:5] # 'Unknown' 은 쓸모없는 데이터 이므로 삭제하자

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [7]:
print('총 샘플의 개수: {}'.format(len(headline)))

총 샘플의 개수: 1324


In [8]:
# 'Unknown' 제거
headline = [word for word in headline if word != 'Unknown']
print('노이즈값 제거 후 샘플의 개수: {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수: 1214


In [9]:
print(headline[:5])

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell', 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.', 'The New Noma, Explained', 'How a Bag of Texas Dirt  Became a Times Tradition', 'Is School a Place for Self-Expression?']


In [10]:
# 데이터 전처리
def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode("utf8").decode("ascii", "ignore")
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [11]:
# 단어 집합을 만들고 크기 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [12]:
# 정수 인코딩 + 하나의 문장을 여러 줄로 분해하여 훈련 데이터 구성

sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [13]:
# 인덱스의 원래 단어를 찾기 위한 배열 정의
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번 단어: offer


In [14]:
# 샘플의 최대 길이
max_len = max(len(l) for l  in sequences)
print('샘플의 최대 길이: {}'.format(max_len))

샘플의 최대 길이: 24


In [15]:
# 패딩 넣기
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
# 특징, 정답 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [17]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [20]:
y = to_categorical(y, num_classes=vocab_size) # 원-핫 인코딩

모델 설계하기

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [22]:
# 다대일 구조의 LSTM

embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y,epochs=200, verbose=2)

Epoch 1/200
244/244 - 5s - 22ms/step - accuracy: 0.0286 - loss: 7.6406
Epoch 2/200
244/244 - 1s - 5ms/step - accuracy: 0.0310 - loss: 7.1162
Epoch 3/200
244/244 - 1s - 5ms/step - accuracy: 0.0329 - loss: 6.9870
Epoch 4/200
244/244 - 2s - 8ms/step - accuracy: 0.0402 - loss: 6.8720
Epoch 5/200
244/244 - 2s - 9ms/step - accuracy: 0.0460 - loss: 6.7270
Epoch 6/200
244/244 - 2s - 10ms/step - accuracy: 0.0495 - loss: 6.5573
Epoch 7/200
244/244 - 1s - 6ms/step - accuracy: 0.0516 - loss: 6.3720
Epoch 8/200
244/244 - 3s - 11ms/step - accuracy: 0.0545 - loss: 6.1778
Epoch 9/200
244/244 - 1s - 6ms/step - accuracy: 0.0601 - loss: 5.9911
Epoch 10/200
244/244 - 2s - 6ms/step - accuracy: 0.0632 - loss: 5.8110
Epoch 11/200
244/244 - 3s - 11ms/step - accuracy: 0.0710 - loss: 5.6445
Epoch 12/200
244/244 - 2s - 9ms/step - accuracy: 0.0732 - loss: 5.4861
Epoch 13/200
244/244 - 1s - 6ms/step - accuracy: 0.0798 - loss: 5.3362
Epoch 14/200
244/244 - 1s - 5ms/step - accuracy: 0.0839 - loss: 5.1902
Epoch 15/20

In [26]:
# 문장 생성 함수 만들기

def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [27]:
# 예측 테스트
print(sentence_generation(model, tokenizer, 'i', 10))
print(sentence_generation(model, tokenizer, 'how', 10))

i cant jump ship from facebook yet midterm fast rude say
how to win an argument about guns life against home to
